In [6]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from scipy.sparse.linalg import svds

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

# Данные

[Источник](https://grouplens.org/datasets/movielens/) исходных данных. 
Данные для скринкаста предобработаны

Извлечем доступные данные о фильмах и о взаимодействиях пользователей с фильмами

In [7]:
ratings = pd.read_csv('ratings_df_sample_2.csv')
movies = pd.read_csv('movies.csv')

In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,54,2,3.00,974918176
1,54,32,5.00,974836809
2,54,47,4.00,974837760
3,54,50,4.00,974837760
4,54,223,5.00,974840217


In [9]:
ratings.describe()

,userId,movieId,rating,timestamp
count,6040099.00,6040099.00,6040099.00,6040099.00
mean,68804.82,4822.96,3.55,1115774334.98
std,40102.24,11368.03,1.00,135843321.30
min,7.00,1.00,0.50,824835410.00
25%,34180.00,919.00,3.00,995660158.00
50%,68914.00,1876.00,4.00,1111706240.00
75%,103281.00,3448.00,4.00,1213151458.50
max,138493.00,81845.00,5.00,1427780469.00


In [10]:
# ratings.info(null_counts=True)

In [11]:
print(ratings.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [12]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


Для удобства, сразу приджоиним фильмы

In [14]:
ratings=(ratings.merge(movies, on='movieId'))

In [15]:
ratings.shape

(6040099, 6)

In [16]:
ratings[:6]

,userId,movieId,rating,timestamp,title,genres
0,54,2,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy
1,91,2,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,116,2,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy
3,124,2,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy
4,129,2,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy
5,131,2,1.00,1238326861,Jumanji (1995),Adventure|Children|Fantasy


In [17]:
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [18]:
ratings.describe()

,userId,movieId,rating,timestamp
count,6040099.00,6040099.00,6040099.00,6040099.00
mean,68804.82,4822.96,3.55,1115774334.98
std,40102.24,11368.03,1.00,135843321.30
min,7.00,1.00,0.50,824835410.00
25%,34180.00,919.00,3.00,995660158.00
50%,68914.00,1876.00,4.00,1111706240.00
75%,103281.00,3448.00,4.00,1213151458.50
max,138493.00,81845.00,5.00,1427780469.00


Для удобства отмасштабируем идентификаторы фильмов таким образом, чтобы они начинались с 0 и заканчивались на n_movies-1

Этот метод позволит находить фильм по индексу матрицы (i-й столбец матрицы это i-й фильм)

In [19]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,54,2,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy
1,91,2,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,116,2,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy
3,124,2,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy
4,129,2,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy


In [20]:
%%time
movies_values = ratings['movieId'].unique()

ratings['movieId'] = ratings['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

CPU times: total: 38.8 s
Wall time: 39.7 s


Также отмасштабируем идентификаторы пользователей таким образом, чтобы они начинались с 0 и заканчивались на n_users-1

Этот метод позволит находить пользовтеля по индексу матрицы (i-я строка матрицы это i-й пользователь)

In [21]:
%%time
users_values = ratings['userId'].unique()

ratings['userId'] = ratings['userId'].apply(lambda f: np.where(users_values == f)[0][0])

CPU times: total: 1min 16s
Wall time: 1min 17s


In [22]:
ratings[:5]

,userId,movieId,rating,timestamp,title,genres
0,0,0,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy
1,1,0,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,2,0,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy
3,3,0,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy
4,4,0,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy


In [23]:
ratings.describe()

,userId,movieId,rating,timestamp
count,6040099.00,6040099.00,6040099.00,6040099.00
mean,9115.02,422.90,3.55,1115774334.98
std,5563.83,283.85,1.00,135843321.30
min,0.00,0.00,0.50,824835410.00
25%,4365.00,165.00,3.00,995660158.00
50%,8679.00,415.00,4.00,1111706240.00
75%,13758.00,646.00,4.00,1213151458.50
max,19999.00,999.00,5.00,1427780469.00


# Контентная рекомендация фильмов

Реализуем алгоритм рекомендации, используя только информацию о фильмах. 

Используя название и жанр фильма, сформируем tf-idf ветора, и для каждого фильма найдем топ 5 похожих фильмов по косинусному расстоянию.

Извлечем фильмы, сделам текстовую фичу test_feature, которую будем кодировать

In [24]:
unique_movies = ratings[['movieId', 'title', 'genres']].drop_duplicates()

In [25]:
unique_movies.shape

(1000, 3)

In [26]:
unique_movies.head()

,movieId,title,genres
0,0,Jumanji (1995),Adventure|Children|Fantasy
9524,1,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
23118,2,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
36942,3,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
51035,4,Clerks (1994),Comedy


In [27]:
unique_movies['genres'] = unique_movies['genres'].apply(lambda x: ' '.join(x.split('|')))

In [28]:
unique_movies['text_feature'] = unique_movies.apply(lambda x: x['title'].lower() + ' ' + x['genres'].lower(), axis = 1)

In [29]:
unique_movies = unique_movies.sort_values('movieId')

In [30]:
unique_movies[:5]

,movieId,title,genres,text_feature
0,0,Jumanji (1995),Adventure Children Fantasy,jumanji (1995) adventure children fantasy
9524,1,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery Sci-Fi Thriller,twelve monkeys (a.k.a. 12 monkeys) (1995) mystery sci-fi thriller
23118,2,Seven (a.k.a. Se7en) (1995),Mystery Thriller,seven (a.k.a. se7en) (1995) mystery thriller
36942,3,"Usual Suspects, The (1995)",Crime Mystery Thriller,"usual suspects, the (1995) crime mystery thriller"
51035,4,Clerks (1994),Comedy,clerks (1994) comedy


Сформируем tf-idf ветора для каждого text_feature  
Как считается [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)

In [31]:
vectorizer = TfidfVectorizer()

In [32]:
tfidf_features = vectorizer.fit_transform(unique_movies['text_feature']).toarray()

In [33]:
tfidf_features.shape

(1000, 1646)

In [34]:
pd.DataFrame(tfidf_features)

,0,1,2,3,4,5,6,7,8,9,...,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.37,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
996,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
997,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
998,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [35]:
[u for u, v in vectorizer.vocabulary_.items() if v == 4]

['12']

# Косинусное расстояние

Рассмотрим функцию для попарных косинусных расстояния массива

Сosine_distances - это единица минус косинус, такая величина берется для того, чтобы выполнялась логика - чем меньше значение тем ближе
вектора

In [36]:
demo_data = np.array([[5,5,5,0,0], [4,1,0,5,3], [1,0,0,5,0], [5,0,5,0,4]])

cosine_distances(demo_data)

array([[0.        , 0.59577396, 0.8867723 , 0.28933095],
       [0.59577396, 0.        , 0.2036092 , 0.4484398 ],
       [0.8867723 , 0.2036092 , 0.        , 0.87929886],
       [0.28933095, 0.4484398 , 0.87929886, 0.        ]])

Проверим что, что на позиции 0 1 действитеьно 1 - косинус

In [37]:
1 - (demo_data[0]*demo_data[1]).sum()/( (demo_data[0]**2).sum() * (demo_data[1]**2).sum())**0.5

0.5957739582727783

Вычислим расстояния

In [38]:
cosine_similarity = cosine_distances(tfidf_features)

In [39]:
cosine_similarity.shape

(1000, 1000)

Зная попарные косинусные расстояния, найдем топ 5 кандидатов для каждого фильма

In [40]:
top=5
movies_sim = []
for i in range(len(cosine_similarity)):
    neighbors = (cosine_similarity[i]).argsort()[1:top+1] # считаем от 1 до top1 + 1, 
    # так как ближайшее расстояние у вектора до самого себя
    movies_sim.append(neighbors)
movies_sim = np.array(movies_sim)

In [41]:
movies_sim.shape

(1000, 5)

Составим датафрейм для того, чтобы понять по описаниям, какие фильмы похожи на какие

In [42]:
movie_id_name = dict(unique_movies.set_index('movieId')['text_feature'])
movies_top_df = pd.DataFrame(movies_sim, columns = ['top1', 'top2', 'top3', 'top4', 'top5'])
movies_top_df['top1'] = movies_top_df['top1'].apply(lambda x: movie_id_name[x])
movies_top_df['top2'] = movies_top_df['top2'].apply(lambda x: movie_id_name[x])
movies_top_df['top3'] = movies_top_df['top3'].apply(lambda x: movie_id_name[x])
movies_top_df['top4'] = movies_top_df['top4'].apply(lambda x: movie_id_name[x])
movies_top_df['top5'] = movies_top_df['top5'].apply(lambda x: movie_id_name[x])

movies_top_df['movieId'] = movies_top_df.index

Посмотрим на полученные рекомендации

In [43]:
movies_recs = (
    unique_movies.merge(
        movies_top_df,
        left_on='movieId', right_on='movieId'
    )
)

In [44]:
movies_recs[:5]

,movieId,title,genres,text_feature,top1,top2,top3,top4,top5
0,0,Jumanji (1995),Adventure Children Fantasy,jumanji (1995) adventure children fantasy,toy story (1995) adventure animation children comedy fantasy,casper (1995) adventure children,"indian in the cupboard, the (1995) adventure children fantasy",babe (1995) children drama,antz (1998) adventure animation children comedy fantasy
1,1,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery Sci-Fi Thriller,twelve monkeys (a.k.a. 12 monkeys) (1995) mystery sci-fi thriller,ocean's twelve (2004) action comedy crime thriller,congo (1995) action adventure mystery sci-fi,strange days (1995) action crime drama mystery sci-fi thriller,12 angry men (1957) drama,outbreak (1995) action drama sci-fi thriller
2,2,Seven (a.k.a. Se7en) (1995),Mystery Thriller,seven (a.k.a. se7en) (1995) mystery thriller,copycat (1995) crime drama horror mystery thriller,"usual suspects, the (1995) crime mystery thriller",strange days (1995) action crime drama mystery sci-fi thriller,congo (1995) action adventure mystery sci-fi,snow white and the seven dwarfs (1937) animation children drama fantasy musical
3,3,"Usual Suspects, The (1995)",Crime Mystery Thriller,"usual suspects, the (1995) crime mystery thriller",copycat (1995) crime drama horror mystery thriller,"net, the (1995) action crime thriller",strange days (1995) action crime drama mystery sci-fi thriller,"negotiator, the (1998) action crime drama mystery thriller",heat (1995) action crime thriller
4,4,Clerks (1994),Comedy,clerks (1994) comedy,i.q. (1994) comedy romance,airheads (1994) comedy,junior (1994) comedy sci-fi,"mask, the (1994) action comedy crime fantasy","flintstones, the (1994) children comedy fantasy"


In [45]:
movies_recs[movies_recs['title'].apply(lambda x: 'Star Wars' in x)]

,movieId,title,genres,text_feature,top1,top2,top3,top4,top5
5,5,Star Wars: Episode IV - A New Hope (1977),Action Adventure Sci-Fi,star wars: episode iv - a new hope (1977) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax
20,20,Star Wars: Episode V - The Empire Strikes Back (1980),Action Adventure Sci-Fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode iv - a new hope (1977) action adventure sci-fi
51,51,Star Wars: Episode I - The Phantom Menace (1999),Action Adventure Sci-Fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode iv - a new hope (1977) action adventure sci-fi
64,64,Star Wars: Episode VI - Return of the Jedi (1983),Action Adventure Sci-Fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi
560,560,Star Wars: Episode II - Attack of the Clones (2002),Action Adventure Sci-Fi IMAX,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi
577,577,Star Wars: Episode III - Revenge of the Sith (2005),Action Adventure Sci-Fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi


# Колаборативная фильтрация

Для оценки прогнозов с помощью колаборативной фильтрации, разобьем датасет на train и test

In [46]:
train_data, test_data = train_test_split(ratings, test_size=0.01)

print('Train shape: {}'.format(train_data.shape))
print('Test shape: {}'.format(test_data.shape))

Train shape: (5979698, 6)
Test shape: (60401, 6)


# Метрика качества

Посчитаем метрику на тесте, если бы мы предсказали всем средний рейтинг

In [47]:
train_data['rating'].mean()

3.5539891813934417

In [48]:
const_default = 3.5
test_data['constant_predict'] = const_default
print(f"RMSE metric: {np.sqrt(mean_squared_error(test_data['constant_predict'], test_data['rating']))}")

RMSE metric: 1.0091980253120807


# Сформируем матрицу user-item

In [49]:
n_users = train_data['userId'].nunique()
n_users

20000

In [50]:
n_movies = train_data['movieId'].nunique()
n_movies

1000

Создаём user-item матрицу – для обучения

Неопределенные ячейки будем заполнять не нулями, а 3.5, так как это средний рейтинг (заполняем пропуски средним)

In [51]:
%%time
train_data_matrix = const_default*np.ones((n_users, n_movies))
for line in train_data.to_dict(orient='records'):
    train_data_matrix[line['userId'], line['movieId']] = line['rating']  

CPU times: total: 17.5 s
Wall time: 17.6 s


In [52]:
train_data_matrix.shape

(20000, 1000)

In [53]:
train_data_matrix

array([[3. , 5. , 4. , ..., 3.5, 3.5, 3.5],
       [3.5, 4. , 4. , ..., 3.5, 3.5, 3.5],
       [2. , 4. , 4.5, ..., 3.5, 3.5, 3.5],
       ...,
       [3.5, 3.5, 3.5, ..., 3.5, 3.5, 3.5],
       [3.5, 3.5, 3.5, ..., 3.5, 3.5, 3.5],
       [3.5, 3.5, 3.5, ..., 3.5, 3. , 3.5]])

# Матрицы попарных косинусных расстояний

In [54]:
%%time
# считаем попарное косинусное расстояние для пользователей (строк матрицы)
user_similarity = cosine_distances(train_data_matrix)

CPU times: total: 22.4 s
Wall time: 6.31 s


In [55]:
# # считаем попарное косинусное расстояние для фильмов (столбцов матрицы)
movie_similarity = cosine_distances(train_data_matrix.T)

In [56]:
user_similarity

array([[0.        , 0.02459358, 0.05513699, ..., 0.01838672, 0.01578322,
        0.018997  ],
       [0.02459358, 0.        , 0.04896937, ..., 0.01594009, 0.01317102,
        0.01792606],
       [0.05513699, 0.04896937, 0.        , ..., 0.04882036, 0.0476033 ,
        0.05330809],
       ...,
       [0.01838672, 0.01594009, 0.04882036, ..., 0.        , 0.0053888 ,
        0.01095453],
       [0.01578322, 0.01317102, 0.0476033 , ..., 0.0053888 , 0.        ,
        0.00754749],
       [0.018997  , 0.01792606, 0.05330809, ..., 0.01095453, 0.00754749,
        0.        ]])

In [57]:
movie_similarity

array([[0.        , 0.03624089, 0.03699741, ..., 0.02398577, 0.02724317,
        0.02513415],
       [0.03624089, 0.        , 0.02555235, ..., 0.02076354, 0.02320895,
        0.02267538],
       [0.03699741, 0.02555235, 0.        , ..., 0.02194172, 0.02295996,
        0.02487794],
       ...,
       [0.02398577, 0.02076354, 0.02194172, ..., 0.        , 0.01163548,
        0.00871386],
       [0.02724317, 0.02320895, 0.02295996, ..., 0.01163548, 0.        ,
        0.01333686],
       [0.02513415, 0.02267538, 0.02487794, ..., 0.00871386, 0.01333686,
        0.        ]])

In [58]:
user_similarity.shape

(20000, 20000)

In [59]:
movie_similarity.shape

(1000, 1000)

# Алгоритм user-based

Для каждого пользователя находим топ 10 ближайших соседей, исключая себя самого (поэтому индекс от единицы до top + 1)

In [60]:
%%time
top=10
top_similar_users = []
for i in range(n_users):
    neighbors = (user_similarity[i]).argsort()[1:top + 1]
    top_similar_users.append(
        train_data_matrix[neighbors]
    )
top_similar_users = np.array(top_similar_users)

CPU times: total: 26.2 s
Wall time: 26.4 s


Количество  пользователей, количество соседей, количество фильмов

In [61]:
top_similar_users.shape

(20000, 10, 1000)

Для каждого пользователя и каждого фильма считаем средний рейтинг у фильма от топ похожих пользователей

Так делам матрицу с предсказанными рейтингами

In [62]:
predicted_ratings_user_based = top_similar_users.mean(1)

In [63]:
predicted_ratings_user_based.shape

(20000, 1000)

Делаем предикт на тестовом датасете, для каждого тестового userId и movieId заполняем предсказанный рейтинг из полученной матрицы

По userId строке и movieId столбцу, для этого и делали смену айдишников

In [64]:
def round_to_nearest_0_5(n):
    return round(n * 2) / 2

In [65]:
test_data['predict_user_based'] = test_data.apply(
    lambda f: round_to_nearest_0_5(predicted_ratings_user_based[f['userId'], f['movieId']]), axis = 1
)

In [66]:
print(f"RMSE metric: {np.sqrt(mean_squared_error(test_data['predict_user_based'], test_data['rating']))}")

RMSE metric: 0.95472282396593


# Алгоритм item-based

Для каждого фильма находим топ 10 ближайших соседей, исключая себя самого (поэтмоу индекс от единицы до top + 1)

In [67]:
top=10
top_similar_ratings = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_ratings.append(
        train_data_matrix.T[neighbors]
    )
    
top_similar_ratings = np.array(top_similar_ratings)

Число фильмов, число соседей, число пользователей

In [68]:
top_similar_ratings.shape

(1000, 10, 20000)

Для каждого пользователя и каждого фильма считаем средний рейтинг от пользователя для топ соседних фильмов 

In [69]:
predicted_ratings_item_based = top_similar_ratings.mean(1).T

In [70]:
predicted_ratings_item_based.shape

(20000, 1000)

Делаем предикт на тестовом датасете, для каждого тестового userId и movieId заполняем предсказанный рейтинг из полученной матрицы

По userId строке и movieId столбцу, для этого и делали смену айдишников

In [71]:
test_data['predict_item_based'] = test_data.apply(
    lambda f: round_to_nearest_0_5(predicted_ratings_item_based[f['userId'], f['movieId']]), axis = 1
)

In [72]:
print(f"RMSE metric: {np.sqrt(mean_squared_error(test_data['predict_item_based'], test_data['rating']))}")

RMSE metric: 0.9608656825417514


Когда у вас много пользователей и мало фильмов, рекомендации, основанные на пользователях (user-based), могут быть более эффективными по сравнению с рекомендациями, основанными на элементах (item-based). Это связано с тем, что user-based рекомендации сосредотачиваются на схожести между пользователями и используют информацию о предпочтениях пользователей, чтобы сделать рекомендации.

В случае user-based рекомендаций для каждого пользователя вычисляется схожесть с другими пользователями на основе их предпочтений по фильмам. Когда множество пользователей больше, выше шанс найти похожие пользовательские профили и предложить релевантные рекомендации.

В отличие от этого, item-based рекомендации фокусируются на схожести между фильмами. Когда у вас мало фильмов, может быть трудно найти достаточно похожих элементов для выработки рекомендаций.

Таким образом, в вашем случае, user-based алгоритм рекомендаций фильмов скорее всего будет более эффективным и произведет более качественные рекомендации. Однако, рекомендации основанные на содержании фильмов (content-based) или гибридные методы, комбинирующие несколько подходов, также могут быть полезными вариантами в ситуации с небольшим количеством фильмов.

# Алгоритм на основе матричного разложения

In [73]:
train_data_matrix.shape

(20000, 1000)

[Документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html) SVDs разложения  
[Документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html) SVD разложения

In [74]:
%%time
# делаем SVD
u, s, vh = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

CPU times: total: 4.55 s
Wall time: 797 ms


In [75]:
np.diag(s).shape

(20, 20)

In [76]:
vh.shape

(20, 1000)

In [77]:
users.shape

(20000, 20)

In [78]:
items.shape

(1000, 20)

In [79]:
test_data

,userId,movieId,rating,timestamp,title,genres,constant_predict,predict_user_based,predict_item_based
2508089,15993,316,1.00,969305029,Taxi Driver (1976),Crime|Drama|Thriller,3.50,3.50,3.50
2339751,18362,289,4.00,1142275124,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,3.50,4.00,3.50
5728391,962,915,3.00,1328672390,"Right Stuff, The (1983)",Drama,3.50,4.00,3.00
5821818,1064,940,3.50,1180957556,Matilda (1996),Children|Comedy|Fantasy,3.50,3.50,3.50
5461028,16859,844,4.00,941218566,Carlito's Way (1993),Crime|Drama,3.50,3.50,3.50
...,...,...,...,...,...,...,...,...,...
3455429,9549,474,2.00,974299215,"Road Warrior, The (Mad Max 2) (1981)",Action|Adventure|Sci-Fi,3.50,3.50,3.50
4623265,11294,663,4.00,1208588918,Sideways (2004),Comedy|Drama|Romance,3.50,3.50,3.50
4658458,6205,669,2.00,1215034941,Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Action|Crime|Drama|Thriller,3.50,3.50,3.50
1684140,7623,190,4.00,1104523342,Face/Off (1997),Action|Crime|Drama|Thriller,3.50,3.50,3.50


In [80]:
test_data['svd_predictions'] = test_data.apply(
    lambda f: round_to_nearest_0_5(np.dot(users[f['userId']], items[f['movieId']])), axis = 1
)

In [81]:
print(f"RMSE metric: {np.sqrt(mean_squared_error(test_data['svd_predictions'], test_data['rating']))}")

RMSE metric: 0.8395692793874393


матрица профилей (эмбедингов) пользователей

In [82]:
users.shape

(20000, 20)

матрица профилей (эмбедингов) фильмов

In [83]:
items.shape

(1000, 20)

посмотрим на соседей фильмов по эмбедингам (так же как по tf-idf)

In [84]:
cosine_similarity_items = cosine_distances(items)

In [85]:
top=5
movies_sim = []
for i in range(len(cosine_similarity_items)):
    neighbors = (cosine_similarity_items[i]).argsort()[1:top+1] # считаем от 1 до top1 + 1, 
    # так как ближайшее расстояние у вектора до самого себя
    movies_sim.append(neighbors)
movies_sim = np.array(movies_sim)

In [86]:
movie_id_name = dict(unique_movies[['movieId', 'text_feature']].values.tolist())
movies_top_df = pd.DataFrame(movies_sim, columns = ['top1', 'top2', 'top3', 'top4', 'top5'])
movies_top_df['top1'] = movies_top_df['top1'].apply(lambda x: movie_id_name[x])
movies_top_df['top2'] = movies_top_df['top2'].apply(lambda x: movie_id_name[x])
movies_top_df['top3'] = movies_top_df['top3'].apply(lambda x: movie_id_name[x])
movies_top_df['top4'] = movies_top_df['top4'].apply(lambda x: movie_id_name[x])
movies_top_df['top5'] = movies_top_df['top5'].apply(lambda x: movie_id_name[x])

movies_top_df = movies_top_df.reset_index()
movies_top_df['movieId'] = movies_top_df['index']

на основе матрично факторизации получили более неочевидные item рекомендации по сравнению с tf-idf

в рекомендациях появился star trek к star wars

In [87]:
movies_recs_matrix = (
    unique_movies.merge(
        movies_top_df,
        left_on='movieId', right_on='movieId'
    )
)

In [88]:
movies_recs_matrix[movies_recs_matrix['title'].apply(lambda x: 'Star Wars' in x)]

,movieId,title,genres,text_feature,index,top1,top2,top3,top4,top5
5,5,Star Wars: Episode IV - A New Hope (1977),Action Adventure Sci-Fi,star wars: episode iv - a new hope (1977) action adventure sci-fi,5,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,raiders of the lost ark (indiana jones and the raiders of the lost ark) (1981) action adventure,indiana jones and the last crusade (1989) action adventure,star wars: episode i - the phantom menace (1999) action adventure sci-fi
20,20,Star Wars: Episode V - The Empire Strikes Back (1980),Action Adventure Sci-Fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,20,star wars: episode iv - a new hope (1977) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,raiders of the lost ark (indiana jones and the raiders of the lost ark) (1981) action adventure,indiana jones and the last crusade (1989) action adventure,star wars: episode i - the phantom menace (1999) action adventure sci-fi
51,51,Star Wars: Episode I - The Phantom Menace (1999),Action Adventure Sci-Fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,51,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,"matrix revolutions, the (2003) action adventure sci-fi thriller imax","matrix reloaded, the (2003) action adventure sci-fi thriller imax",star trek: generations (1994) adventure drama sci-fi
64,64,Star Wars: Episode VI - Return of the Jedi (1983),Action Adventure Sci-Fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,64,star wars: episode iv - a new hope (1977) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,indiana jones and the last crusade (1989) action adventure,raiders of the lost ark (indiana jones and the raiders of the lost ark) (1981) action adventure,star wars: episode i - the phantom menace (1999) action adventure sci-fi
560,560,Star Wars: Episode II - Attack of the Clones (2002),Action Adventure Sci-Fi IMAX,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,560,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,"matrix revolutions, the (2003) action adventure sci-fi thriller imax","matrix reloaded, the (2003) action adventure sci-fi thriller imax",x-men: the last stand (2006) action sci-fi thriller
577,577,Star Wars: Episode III - Revenge of the Sith (2005),Action Adventure Sci-Fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,577,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode i - the phantom menace (1999) action adventure sci-fi,"matrix reloaded, the (2003) action adventure sci-fi thriller imax","matrix revolutions, the (2003) action adventure sci-fi thriller imax",x-men: the last stand (2006) action sci-fi thriller


tf-idf

In [89]:
movies_recs[movies_recs['title'].apply(lambda x: 'Star Wars' in x)]

,movieId,title,genres,text_feature,top1,top2,top3,top4,top5
5,5,Star Wars: Episode IV - A New Hope (1977),Action Adventure Sci-Fi,star wars: episode iv - a new hope (1977) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax
20,20,Star Wars: Episode V - The Empire Strikes Back (1980),Action Adventure Sci-Fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode iv - a new hope (1977) action adventure sci-fi
51,51,Star Wars: Episode I - The Phantom Menace (1999),Action Adventure Sci-Fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode iv - a new hope (1977) action adventure sci-fi
64,64,Star Wars: Episode VI - Return of the Jedi (1983),Action Adventure Sci-Fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi
560,560,Star Wars: Episode II - Attack of the Clones (2002),Action Adventure Sci-Fi IMAX,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi
577,577,Star Wars: Episode III - Revenge of the Sith (2005),Action Adventure Sci-Fi,star wars: episode iii - revenge of the sith (2005) action adventure sci-fi,star wars: episode i - the phantom menace (1999) action adventure sci-fi,star wars: episode vi - return of the jedi (1983) action adventure sci-fi,star wars: episode ii - attack of the clones (2002) action adventure sci-fi imax,star wars: episode v - the empire strikes back (1980) action adventure sci-fi,star wars: episode iv - a new hope (1977) action adventure sci-fi


: 